# **<p style = "text-align: center">NHẬP MÔN KHOA HỌC DỮ LIỆU</p>**
## **<p style = "text-align: center">Đồ án Cuối kì</p>**
### <p style = "text-align: center"> <b>Đề tài:</b> Quy trình xử lí, phân tích và huấn luyện mô hình cho dự đoán game trên Steam (Steam Game Rating)</p>
**Sinh viên thực hiện:**

| Mã số sinh viên | Họ và tên |
| --- | --- |
| 21120576 | Trần Đình Nhật Trí |
| 21120590 | Nguyễn Thủy Uyên |
| 21120501 | Nguyễn Ngọc Gia Minh |

<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Collect Data
</div>

## Import thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np
import urllib.robotparser
import time
from bs4 import BeautifulSoup
import requests
from fractions import Fraction

### Check dữ liệu trên trang này có hợp pháp khi thu thập

In [3]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://store.steampowered.com/robots.txt')
rp.read()

url = 'https://store.steampowered.com/search/?category1=998&ndl=1'
rp.can_fetch('*', url)

True

Dữ liệu hợp pháp!

### Crawl data với các thuộc tính

'''  
title  
genre  
tags  
category  
is_dlc  
is_mature  
franchise  
release date  
developer  
publisher  
languages  
overwhelmingly positive count  
user reviews count  
negative count  
price  
'''

In [3]:
# Title = []  
# Genre = []  
# Tags = []   
# withDLC = []  
# isMature = [] 
# Franchise = []
# ReleaseDate = []  
# Developer = []  
# Publisher = []
# Languages = []
# PositiveReviews = []
# TotalReviews = []
# NegativeReviews = []  
# OriginalPrice = []
# DiscountPercent = []

In [4]:
def crawl_one_game(item_url):
    response = requests.get(item_url)
    content = response.text
    if response.status_code == 200:
        soup = BeautifulSoup(content, 'html.parser')
        
        # set all attribute is NaN
        title = np.nan 
        genre = np.nan
        tags = np.nan
        with_dlc = np.nan 
        is_mature = np.nan
        franchise = np.nan
        release_date = np.nan
        developer = np.nan
        publisher = np.nan
        languages = np.nan
        positive_reviews = np.nan
        total_reviews = np.nan
        negative_reviews = np.nan

        # Check if a dlc
        is_dlc = soup.find('div',attrs={'class':'game_area_bubble game_area_dlc_bubble'})
        # Check if a soundtrack
        is_soundtrack = soup.find('div',attrs={'class':'game_area_bubble game_area_soundtrack_bubble'})
        
        if is_dlc or is_soundtrack:
            return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews

        # Check game has DLC
        try:
            with_dlc_tmp = soup.find('div',attrs={'id':'gameAreaDLCSection'})
            with_dlc = (True if with_dlc_tmp else False) 
        except:
            pass

        # Title
        try:
            title = soup.find('div',attrs={'class':'apphub_AppName'}).string.strip() 
        except:
            pass

        # Genere
        try:
            genre = [] # A game can have more than one genre
            genre_tmp = soup.find('div',attrs={'id':'genresAndManufacturer'}).find('span')
            for i in genre_tmp.find_all('a'):
                genre.append(i.text.strip())
                
            genre = ", ".join(genre)
        except:
            genre = np.nan
            pass

        # Tag
        try:
            tags = [] # A game can have more than one tag
            for i in soup.find('div',attrs={'class':'glance_tags popular_tags'}).find_all('a'):         
                if i.text.strip() != '+':
                    tags.append(i.text.strip())
                    
            tags = ", ".join(tags)
        except:
            tags = np.nan
            pass

        # Check Mature
        try:
            is_mature_tmp = soup.find('div',attrs={'id':'game_area_content_descriptors'})
            is_mature = (True if is_mature_tmp else False)
        except:
            pass

        # Developer, Publisher, Franchise
        try:
            for i in soup.find('div',attrs={'id':'genresAndManufacturer'}).find_all('div',attrs={'class':'dev_row'}):
                if i.find('b').text == 'Developer:':
                    developer = i.find('a').text
                elif i.find('b').text == 'Publisher:':
                    publisher = i.find('a').text
                elif i.find('b').text == 'Franchise:':
                    franchise = i.find('a').text
        except:
            pass

        # Release Date
        try:
            release_date = soup.find('div',attrs={'class':'date'}).string.strip()
        except:
            pass 

        # Languages
        try:
            languages = [] # A game can have more than one language
            for i in soup.find('div',attrs={'id':'languageTable'}).find_all('td',attrs={'class':'ellipsis'}):   
                languages.append(i.text.strip())
            
            languages = ", ".join(languages)
        except:
            languages = np.nan
            pass

        # All review
        try:
            total_reviews = soup.find('label',attrs={'for':'review_type_all'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
            total_reviews = int(total_reviews.replace(')','').replace('(',''))
        except:
            pass

        # Negative review
        try:
            negative_reviews = soup.find('label',attrs={'for':'review_type_negative'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
            negative_reviews = int(negative_reviews.replace(')','').replace('(',''))
        except:
            pass

        # Positive review
        try:
            positive_reviews = soup.find('label',attrs={'for':'review_type_positive'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
            positive_reviews = int(positive_reviews.replace(')','').replace('(',''))
        except:
            pass

        return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews

In [5]:
data = pd.DataFrame()

In [6]:
flag_page=True
index=1

# get the games from all the pages and not give it a range because its very dynamic
while(flag_page and index!=200):
    Title = []  
    Genre = []  
    Tags = []   
    withDLC = []  
    isMature = [] 
    Franchise = []
    ReleaseDate = []  
    Developer = []  
    Publisher = []
    Languages = []
    PositiveReviews = []
    TotalReviews = []
    NegativeReviews = []  
    OriginalPrice = []
    DiscountPercent = []
    print(index)
    
    # Add page parameter
    parameter = '&page='+str(index)
    
    # Send a request to the website
    web_response = requests.get(url+parameter)
        
    # Get the content of the request
    web_content = web_response.text
    
    if web_response.status_code == 200: # request successfully
        web_soup = BeautifulSoup(web_content, 'html.parser')
        
        item_link = []
        
        # Check if the last page
        if index != 1:
            # Find the number of buttons on the page and return False if it has 2
            button = web_soup.find_all('a',attrs={'class':'pagebtn'})
            if len(button)!= 2 :
                flag_page = False
    
        # Add each link to links
        links = web_soup.find('div', attrs={'id': 'search_resultsRows'})
        if links is None:
            index += 1
            continue
    
        for link in links.find_all('a'):
            # Price with discount
            with_discount = link.find('div',attrs={'class':'discount_block search_discount_block'})
            # Price without discount
            without_discount = link.find('div',attrs={'class':'discount_block search_discount_block no_discount'})
            # Free price
            free_price = link.find('div',attrs={'class':'discount_block no_discount search_discount_block'})
            
            if with_discount:
                DiscountPercent.append(with_discount.find('div',attrs={'class':'discount_pct'}).text)
                OriginalPrice.append(with_discount.find('div',attrs={'class':'discount_original_price'}).text)      
            elif without_discount:
                DiscountPercent.append(np.nan)
                OriginalPrice.append(without_discount.find('div',attrs={'class':'discount_final_price'}).text)
            elif free_price:
                DiscountPercent.append(np.nan)
                OriginalPrice.append(free_price.find('div',attrs={'class':'discount_final_price free'}).text)
            else:
                break
        
            item_link.append(link['href'])
            
        # Add attribute of each game
        for item in item_link:    
            title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews = crawl_one_game(item)              
            
            # Add values into attributes
            Title.append(title)  
            Genre.append(genre)
            Tags.append(tags)  
            withDLC.append(with_dlc)
            isMature.append(is_mature)
            Franchise.append(franchise)
            ReleaseDate.append(release_date) 
            Developer.append(developer)
            Publisher.append(publisher)
            Languages.append(languages)
            PositiveReviews.append(positive_reviews)
            TotalReviews.append(total_reviews)
            NegativeReviews.append(negative_reviews)
    crawled_data = pd.DataFrame({"Title": Title,
                     "Genre": Genre,
                     "Tags": Tags,
                     "withDLC": withDLC,
                     "isMature": isMature,
                     "Franchise": Franchise,
                     "ReleaseDate": ReleaseDate,
                     "Developer": Developer,
                     "Publisher": Publisher,
                     "Languages": Languages,
                     "PositiveReviews": PositiveReviews,
                     "TotalReviews": TotalReviews,
                     "NegativeReviews": NegativeReviews,
                     "OriginalPrice": OriginalPrice,
                     "DiscountPercent": DiscountPercent})   
                     
    data = pd.concat([data, crawled_data], axis = 0) 
    # Increase page index
    index += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [7]:
# data = pd.DataFrame({"Title": Title,
#                      "Genre": Genre,
#                      "Tags": Tags,
#                      "withDLC": withDLC,
#                      "isMature": isMature,
#                      "Franchise": Franchise,
#                      "ReleaseDate": ReleaseDate,
#                      "Developer": Developer,
#                      "Publisher": Publisher,
#                      "Languages": Languages,
#                      "PositiveReviews": PositiveReviews,
#                      "TotalReviews": TotalReviews,
#                      "NegativeReviews": NegativeReviews,
#                      "OriginalPrice": OriginalPrice,
#                      "DiscountPercent": DiscountPercent})

data

,Title,Genre,Tags,withDLC,isMature,Franchise,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,OriginalPrice,DiscountPercent
0,Endless Monday: Dreams and Deadlines,"Adventure, Indie","Visual Novel, Choose Your Own Adventure, Inter...",False,False,Endless Monday,"4 May, 2023",hcnone,hcnone,"English, Japanese",737,737,0.0,"142.000,00₫",NaN
1,Senren＊Banka,"Adventure, Casual","Visual Novel, NSFW, Casual, Dating Sim, Anime,...",False,True,NaN,"14 Feb, 2020",YUZUSOFT,HIKARI FIELD,"English, Japanese, Simplified Chinese, Traditi...","(17,148)","(17,298)",150.0,"280.000,00₫",NaN
2,A Short Hike,"Adventure, Indie","Exploration, Adventure, Relaxing, Cute, Indie,...",False,False,NaN,"30 Jul, 2019",adamgryu,adamgryu,"English, French, Spanish - Latin America, Japa...","(14,772)","(14,884)",112.0,"100.000,00₫",NaN
3,Aseprite,"Animation & Modeling, Design & Illustration, G...","Pixel Graphics, Design & Illustration, Animati...",False,False,NaN,"22 Feb, 2016",David Capello,Igara Studio,English,"(15,565)","(15,693)",128.0,"188.000,00₫",NaN
4,Cats Hidden in Paris,"Casual, Indie, Free to Play","Cats, Hidden Object, Puzzle, Wholesome, Relaxi...",True,False,Travellin Cats,"16 Jul, 2023",Travellin Cats,Travellin Cats,"English, French, Italian, German, Spanish - Sp...","(12,039)","(12,144)",105.0,Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"74.000,00₫",NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"102.000,00₫",NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"125.000,00₫",-50%
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"192.000,00₫",NaN


In [8]:
data.to_csv('./data/steam_game.csv', index = False)